<a href="https://colab.research.google.com/github/azzindani/01_Data_Analytic/blob/main/Adidas_US_Draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 00 Importing Modules & Mounting Drive

In [1]:
!pip install pandas-profiling
!pip install streamlit
!npm install localtunnel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.6/357.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.7/102.7 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.5/679.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 30.2 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27081 sha256=a7846a93bc8aad02f8d17871af57350ae131b6679f57af94dbe887a6da3f0845
  Stored in directory: /root/.cache/pip/wheels/dd/91/29/a79cecb328d01739e64017b6fb9a1ab9d8cb1853098ec5966d
Successfully built htmlmin
  Attempting unins

In [2]:
import pathlib
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
import os
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from ydata_profiling import ProfileReport
from tabulate import tabulate
from time import sleep

%matplotlib inline

In [3]:
'''from google.colab import drive
drive.mount('/content/drive') #'''

"from google.colab import drive\ndrive.mount('/content/drive') #"

In [4]:
'''MAIN_PATH = str(pathlib.Path().resolve())
WORK_PATH = MAIN_PATH + '/drive/MyDrive/Workspace'
SOURCE_PATH = WORK_PATH + '/00_Data_Source'
CACHE_PATH = WORK_PATH + '/00_Cache_Data' #'''

"MAIN_PATH = str(pathlib.Path().resolve())\nWORK_PATH = MAIN_PATH + '/drive/MyDrive/Workspace'\nSOURCE_PATH = WORK_PATH + '/00_Data_Source'\nCACHE_PATH = WORK_PATH + '/00_Cache_Data' #"

## 01 Choosing & Importing Dataset

### 01.01 Importing Main Data

In [5]:
dataset_path = 'https://raw.githubusercontent.com/azzindani/00_Data_Source/main/Adidas_US_Sales.csv'
df = pd.read_csv(dataset_path)
df.head(2)

,Retailer,Retailer ID,Invoice Date,Region,State,City,Product,Price per Unit,Units Sold,Total Sales,Operating Profit,Operating Margin,Sales Method
0,Foot Locker,1185732,2020-01-01,Northeast,New York,New York,Men's Street Footwear,50.0,1200,600000.0,300000.0,0.5,In-store
1,Foot Locker,1185732,2020-01-02,Northeast,New York,New York,Men's Athletic Footwear,50.0,1000,500000.0,150000.0,0.3,In-store


### 01.02 Importing Geo Data

In [6]:
gdf = gpd.read_file('https://raw.githubusercontent.com/PublicaMundi/MappingAPI/master/data/geojson/us-states.json')
gdf.head(2) #'''

,id,name,density,geometry
0,01,Alabama,94.650,"POLYGON ((-87.35930 35.00118, -85.60667 34.984..."
1,02,Alaska,1.264,"MULTIPOLYGON (((-131.60202 55.11798, -131.5691..."


In [7]:
gdf = gdf.rename(columns = {'name' : 'State'})
gdf = gdf[['State', 'geometry']]
gdf.head(3)

,State,geometry
0,Alabama,"POLYGON ((-87.35930 35.00118, -85.60667 34.984..."
1,Alaska,"MULTIPOLYGON (((-131.60202 55.11798, -131.5691..."
2,Arizona,"POLYGON ((-109.04250 37.00026, -109.04798 31.3..."


### 01.03 Importing Additional Data (for enrichment)

## 02 Checking Data

In [8]:
df.shape

(9648, 13)

In [9]:
df.columns

Index(['Retailer', 'Retailer ID', 'Invoice Date', 'Region', 'State', 'City',
       'Product', 'Price per Unit', 'Units Sold', 'Total Sales',
       'Operating Profit', 'Operating Margin', 'Sales Method'],
      dtype='object')

In [10]:
df.nunique()

Retailer               6
Retailer ID            4
Invoice Date         724
Region                 5
State                 50
City                  52
Product                6
Price per Unit       139
Units Sold           361
Total Sales         3464
Operating Profit    6298
Operating Margin     110
Sales Method           3
dtype: int64

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9648 entries, 0 to 9647
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Retailer          9648 non-null   object 
 1   Retailer ID       9648 non-null   int64  
 2   Invoice Date      9648 non-null   object 
 3   Region            9648 non-null   object 
 4   State             9648 non-null   object 
 5   City              9648 non-null   object 
 6   Product           9648 non-null   object 
 7   Price per Unit    9648 non-null   float64
 8   Units Sold        9648 non-null   int64  
 9   Total Sales       9648 non-null   float64
 10  Operating Profit  9648 non-null   float64
 11  Operating Margin  9648 non-null   float64
 12  Sales Method      9648 non-null   object 
dtypes: float64(4), int64(2), object(7)
memory usage: 980.0+ KB


## 03 Data Structuring

### 03.01 Selecting & Dropping Variables

In [12]:
df = df.drop(['Retailer ID', 'City'], axis = 1)
df.head(3)

,Retailer,Invoice Date,Region,State,Product,Price per Unit,Units Sold,Total Sales,Operating Profit,Operating Margin,Sales Method
0,Foot Locker,2020-01-01,Northeast,New York,Men's Street Footwear,50.0,1200,600000.0,300000.0,0.50,In-store
1,Foot Locker,2020-01-02,Northeast,New York,Men's Athletic Footwear,50.0,1000,500000.0,150000.0,0.30,In-store
2,Foot Locker,2020-01-03,Northeast,New York,Women's Street Footwear,40.0,1000,400000.0,140000.0,0.35,In-store


### 03.02 Cleaning Text Object

#### 03.02.01 Convert Header to Proper Text

In [13]:
for x in df.columns:
  y = x.title()
  df = df.rename(columns = {x : y})

#### 03.02.02 Strip Abnormal Spaces

In [14]:
for x in df.columns:
  if df[x].dtypes == 'object':
    try:
      df[x] = df[x].str.strip()
    except:
      pass

#### 03.02.03 Convert Object Content to Proper Text

In [15]:
for x in df.columns:
  if df[x].dtypes == 'object':
    for a in df[x].unique():
      b = a.title()
      df[x] = df[x].replace(a, b)
  else:
    pass

### 03.03 Coverting Data

#### 03.03.01 Convert to date

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9648 entries, 0 to 9647
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Retailer          9648 non-null   object 
 1   Invoice Date      9648 non-null   object 
 2   Region            9648 non-null   object 
 3   State             9648 non-null   object 
 4   Product           9648 non-null   object 
 5   Price Per Unit    9648 non-null   float64
 6   Units Sold        9648 non-null   int64  
 7   Total Sales       9648 non-null   float64
 8   Operating Profit  9648 non-null   float64
 9   Operating Margin  9648 non-null   float64
 10  Sales Method      9648 non-null   object 
dtypes: float64(4), int64(1), object(6)
memory usage: 829.2+ KB


In [17]:
df['Invoice Date'] = pd.to_datetime(df['Invoice Date'])

#### 03.03.02 Convert to integer

#### 03.03.03 Convert to Object (if necessary)

#### 03.03.04 Replace 0 to Nan (if necessary)

In [18]:
#df = df.replace(0, np.nan)

## 04 Data Cleaning

### 04.01 Replacing Variable Component

### 04.02 Add New Variable

### 04.03 Inaccuracies

### 04.04 Handling Outliers
##### Data Distribution Check (Before)

In [19]:
df.head(3)

,Retailer,Invoice Date,Region,State,Product,Price Per Unit,Units Sold,Total Sales,Operating Profit,Operating Margin,Sales Method
0,Foot Locker,2020-01-01,Northeast,New York,Men'S Street Footwear,50.0,1200,600000.0,300000.0,0.50,In-Store
1,Foot Locker,2020-01-02,Northeast,New York,Men'S Athletic Footwear,50.0,1000,500000.0,150000.0,0.30,In-Store
2,Foot Locker,2020-01-03,Northeast,New York,Women'S Street Footwear,40.0,1000,400000.0,140000.0,0.35,In-Store


In [20]:
'''fig = px.histogram(
    df,
    x = 'Total Sales',
    y = 'Units Sold',
    color = 'Retailer',
    marginal = 'box',
    hover_data = df.columns
)

fig.show() #'''

"fig = px.histogram(\n    df,\n    x = 'Total Sales',\n    y = 'Units Sold',\n    color = 'Retailer',\n    marginal = 'box',\n    hover_data = df.columns\n)\n\nfig.show() #"

In [21]:
'''fig = px.scatter(
    df,
    x = 'Total Sales',
    y = 'Units Sold',
    color = 'Retailer',
    hover_data = df.columns
)

fig.show() #'''

"fig = px.scatter(\n    df,\n    x = 'Total Sales',\n    y = 'Units Sold',\n    color = 'Retailer',\n    hover_data = df.columns\n)\n\nfig.show() #"

#### 04.04.01 Using IQR (Inter Quantile Range)

In [22]:
def iqr_thres(dataframe, column, th1 = 0.25, th3 = 0.75):
    quartile1 = dataframe[column].quantile(th1)
    quartile3 = dataframe[column].quantile(th3)
    iqr = quartile3 - quartile1
    upper_limit = quartile3 + 1.5 * iqr
    lower_limit = quartile1 - 1.5 * iqr

    return lower_limit, upper_limit

In [23]:
def check_outliers_iqr(dataframe, column):
    lower_limit, upper_limit = iqr_thres(dataframe, column)
    if dataframe[(dataframe[column] > upper_limit) | (dataframe[column] < lower_limit)].any(axis = None):
        return True
    else:
        return False

In [24]:
def replace_iqr(dataframe, columns, th1 = 0.25, th3 = 0.75, replace = False):
  data = []

  for column in columns:
    if df[column].dtypes == 'int64' or df[column].dtypes == 'float64':
      if column != 'Outcome':
        outliers_ = check_outliers_iqr(df, column)
        count = None
        lower_limit, upper_limit = iqr_thres(dataframe, column, th1, th3)

        if outliers_:
          count = dataframe[(dataframe[column] > upper_limit) | (dataframe[column] < lower_limit)][column].count()
          if replace:
            if lower_limit < 0:
              dataframe.loc[(dataframe[column] > upper_limit), column] = upper_limit
            else:
              dataframe.loc[(dataframe[column] < lower_limit), column] = lower_limit
              dataframe.loc[(dataframe[column] > upper_limit), column] = upper_limit
        outliers_status = check_outliers_iqr(df, column)
        data.append([outliers_, outliers_status, count, column, lower_limit, upper_limit ])

  table = tabulate(data, headers = ['Outliers (Previously)', 'Outliers', 'Count', 'Column', 'Lower Limit', 'Upper Limit'], tablefmt = 'rst', numalign = 'right')
  print('Removing Outliers using IQR')
  print(table)

In [25]:
'''replace_iqr(
    dataframe = df,
    columns = [],
    replace = True
) #'''

'replace_iqr(\n    dataframe = df,\n    columns = [],\n    replace = True\n) #'

#### 04.04.02 Using Standard Deviation

In [26]:
def std_thres(dataframe, column):
    upper_limit = dataframe[column].mean() + 3 * dataframe[column].std()
    lower_limit = dataframe[column].mean() - 3 * dataframe[column].std()

    return lower_limit, upper_limit

In [27]:
def check_outliers_std(dataframe, column):
    lower_limit, upper_limit = iqr_thres(dataframe, column)
    if dataframe[(dataframe[column] > upper_limit) | (dataframe[column] < lower_limit)].any(axis = None):
        return True
    else:
        return False

In [28]:
def replace_std(dataframe, columns, replace = False):
  data = []

  for column in columns:
    if df[column].dtypes == 'int64' or df[column].dtypes == 'float64':
      if column != 'Outcome':
        outliers_ = check_outliers_std(dataframe, column)
        count = None
        lower_limit, upper_limit = std_thres(dataframe, column)

        if outliers_:
          count = dataframe[(dataframe[column] > upper_limit) | (dataframe[column] < lower_limit)][column].count()
          if replace:
            if lower_limit < 0:
              dataframe.loc[(dataframe[column] > upper_limit), column] = upper_limit
            else:
              dataframe.loc[(dataframe[column] < lower_limit), column] = lower_limit
              dataframe.loc[(dataframe[column] > upper_limit), column] = upper_limit
        outliers_status = check_outliers_std(dataframe, column)
        data.append([outliers_, outliers_status, count, column, lower_limit, upper_limit])

  table = tabulate(data, headers = ['Outlier (Previously)', 'Outliers', 'Count', 'Column', 'Lower Limit', 'Upper Limit'], tablefmt = 'rst', numalign = 'right')
  print('Removing Outliers using 3 Standard Deviation')
  print(table)

In [29]:
'''replace_iqr(
    dataframe = df,
    columns = [],
    replace = False
) #'''

'replace_iqr(\n    dataframe = df,\n    columns = [],\n    replace = False\n) #'

### 04.05 Handling Missing / Zeros / Null
##### Filling missing value (numerical only) is better using median than mean or mode

#### 04.05.01 Detecting Zero Value

In [30]:
zero_columns = []

for x in df.columns:
  if df[x].dtypes == 'int64' or df[x].dtypes == 'float64':
    if (df[x] == 0).sum() != 0:
      print(x, ':', str((df[x] == 0).sum()))
      zero_columns.append(x)

Units Sold : 4
Total Sales : 4
Operating Profit : 4


#### 04.05.02 Detecting Nan / Non Available Value

In [31]:
nan_columns = []

for x in df.columns:
  if df[x].dtypes == 'object':
    if (df[x] == np.nan).sum() or (df[x] == '-').sum() != 0:
      print(x, ':', str((df[x] == np.nan).sum()))
      nan_columns.append(x)

#### 04.05.03 Replacing Zero with Mean (for numerical value if median value == 0), if necessary

In [32]:
'''cols = [
] #'''

'cols = [\n] #'

In [33]:
'''df = df.replace(0, np.nan)

for x in cols:
  df[x] = df[x].fillna(df[x].mean()) #'''

'df = df.replace(0, np.nan)\n\nfor x in cols:\n  df[x] = df[x].fillna(df[x].mean()) #'

#### 04.05.04 Replacing Zero with Median (for numerical value if median value != 0), if necessary

In [34]:
'''cols = [
] #'''

'cols = [\n] #'

In [35]:
'''df = df.replace(0, np.nan)

for x in cols:
  df[x] = df[x].fillna(df[x].median()) #'''

'df = df.replace(0, np.nan)\n\nfor x in cols:\n  df[x] = df[x].fillna(df[x].median()) #'

#### 04.05.05 Replacing Zero with Mode (for categorical / object value), if necessary

In [36]:
'''cols = [
] #'''

'cols = [\n] #'

In [37]:
'''df = df.replace(0, np.nan)

for x in cols:
  df[x] = df[x].fillna(df[x].mode()) #'''

'df = df.replace(0, np.nan)\n\nfor x in cols:\n  df[x] = df[x].fillna(df[x].mode()) #'

### 04.06 Handling Incomplete Data

### 04.07 Handling Data Biases

### 04.08 Handling Duplicates

In [38]:
#df = df.drop_duplicates()

##### Data Distribution Check (After)

In [39]:
'''fig = px.histogram(
    df,
    x = 'Total Sales',
    y = 'Units Sold',
    color = 'Retailer',
    marginal = 'box',
    hover_data = df.columns
)

fig.show() #'''

"fig = px.histogram(\n    df,\n    x = 'Total Sales',\n    y = 'Units Sold',\n    color = 'Retailer',\n    marginal = 'box',\n    hover_data = df.columns\n)\n\nfig.show() #"

In [40]:
'''fig = px.scatter(
    df,
    x = 'Total Sales',
    y = 'Units Sold',
    color = 'Retailer',
    hover_data = df.columns
)

fig.show() #'''

"fig = px.scatter(\n    df,\n    x = 'Total Sales',\n    y = 'Units Sold',\n    color = 'Retailer',\n    hover_data = df.columns\n)\n\nfig.show() #"

## 05 Enriching Data
#### take other dataset, inside or outside from related dataset / business

## 06 Data Validation
#### Verifying consistency, quality, and security of data

## 07 Exploration Data Analysis (Univariate)

### 07.01 Generate Data Profile

In [41]:
'''data_profile = ProfileReport(
    df,
    title = 'Adidas US Sales',
    dataset = {
        #'description' : 'This profiling report was generated for Analytics Vidhya Blog',
        #'copyright_holder' : 'Analytics Vidhya',
        #'copyright_year' : '2021',
        #'url' :  'https://www.analyticsvidhya.com/blog/',
    },
    correlations = {
        'pearson' : {'calculate' : True},
        'spearman' : {'calculate' : True},
        'kendall' : {'calculate' : True},
        'phi_k' : {'calculate' : True},
    },
)

data_profile #'''

"data_profile = ProfileReport(\n    df,\n    title = 'Adidas US Sales',\n    dataset = {\n        #'description' : 'This profiling report was generated for Analytics Vidhya Blog',\n        #'copyright_holder' : 'Analytics Vidhya',\n        #'copyright_year' : '2021',\n        #'url' :  'https://www.analyticsvidhya.com/blog/',\n    },\n    correlations = {\n        'pearson' : {'calculate' : True},\n        'spearman' : {'calculate' : True},\n        'kendall' : {'calculate' : True},\n        'phi_k' : {'calculate' : True},\n    },\n)\n\ndata_profile #"

In [42]:
df.head(3)

,Retailer,Invoice Date,Region,State,Product,Price Per Unit,Units Sold,Total Sales,Operating Profit,Operating Margin,Sales Method
0,Foot Locker,2020-01-01,Northeast,New York,Men'S Street Footwear,50.0,1200,600000.0,300000.0,0.50,In-Store
1,Foot Locker,2020-01-02,Northeast,New York,Men'S Athletic Footwear,50.0,1000,500000.0,150000.0,0.30,In-Store
2,Foot Locker,2020-01-03,Northeast,New York,Women'S Street Footwear,40.0,1000,400000.0,140000.0,0.35,In-Store


## 08 Store Data to Google Drive

In [43]:
'''cache_dir = CACHE_PATH + '/nyc_property_cache.csv'
print(cache_dir)
df.to_csv(cache_dir, index = False)

sleep(60) #'''

"cache_dir = CACHE_PATH + '/nyc_property_cache.csv'\nprint(cache_dir)\ndf.to_csv(cache_dir, index = False)\n\nsleep(60) #"

## 09 Table, Chart Drafting & Testing

In [102]:
fig = px.colors.sequential.swatches_continuous()
fig.show()

In [51]:
colors_1 = px.colors.sequential.Reds
colors_2 = px.colors.sequential.RdBu
explode = tuple([0.015] * 50)
latitude = 37.09024
longitude = -73.935242
chart_theme = 'plotly_dark'
streamlit_theme = 'streamlit'
margin = {'r' : 20, 't' : 40, 'l' : 20, 'b' : 10}
cmap = 'magma_r'
title_x = 0
title_font_size = 18

In [95]:
chart_df_1 = df.groupby(by = 'Retailer', as_index = False,)[['Total Sales']].sum()

title = 'Sales Value by Retailer'
fig = px.bar(
  chart_df_1,
  y = 'Retailer',
  x = 'Total Sales',
  title = title,
  color_discrete_sequence = colors_1,
  text_auto = '$,.0f',
)

fig.update_traces(
    textfont_size = 12,
    textangle = 0,
    textposition = 'inside',
    cliponaxis = False
)

fig.update_xaxes(showspikes = True)
fig.update_yaxes(showspikes = True)
fig.update_layout(
  height = 300,
  margin = margin,
  title = title,
  titlefont = dict(size = title_font_size),
  title_x = title_x,
)
fig.show()

In [45]:
map_df = df.groupby(['State'])['Total Sales'].sum()
map_df = map_df.reset_index()
map_df = map_df.rename(columns = {'Total Sales' : 'Sales Value (USD)'})
map_df = gdf.merge(map_df, on = 'State')

var_label = 'State'
var_number = 'Sales Value (USD)'

map_df = map_df.set_index(var_label)
map_df.head(3)

,geometry,Sales Value (USD)
State,,
Alabama,"POLYGON ((-87.35930 35.00118, -85.60667 34.984...",17633424.0
Alaska,"MULTIPOLYGON (((-131.60202 55.11798, -131.5691...",14753103.0
Arizona,"POLYGON ((-109.04250 37.00026, -109.04798 31.3...",15782221.0


In [46]:
fig = px.choropleth_mapbox(
    data_frame = map_df,
    geojson = map_df.geometry,
    locations = map_df.index,
    color = var_number,
    color_continuous_scale = colors_1,
    #range_color = (0, 10),
    opacity = 0.5,
    center = {'lat' : latitude, 'lon' : longitude},
    mapbox_style = 'carto-positron',
    zoom = 2.5,
)

fig.update_geos(
    fitbounds = 'locations',
    visible = False,
)
fig.update_layout(
    height = 600,
)
fig.show()

In [88]:
chart_df_4 = df.groupby(by = ['Product'], as_index = False,)[['Total Sales', 'Operating Profit']].sum()

title = 'Sales & Operating Profit by Product'
#st.subheader(title)
fig = go.Figure(data = [
  go.Bar(
    name = 'Total Sales',
    x = chart_df_4['Product'],
    y = chart_df_4['Total Sales'],
    text = ['${:,.0f}'.format(x) for x in chart_df_4['Total Sales']],
    marker = {'color': colors_2[0]}),
  go.Bar(
    name = 'Operating Margin',
    x = chart_df_4['Product'],
    y = chart_df_4['Operating Profit'],
    text = ['${:,.0f}'.format(x) for x in chart_df_4['Operating Profit']],
    marker = {'color': colors_2[2]}),
])

fig.update_layout(
    template = chart_theme,
    #paper_bgcolor = 'LightSteelBlue',
    height = 400,
    margin = margin,
    title = title,
    titlefont = dict(size = title_font_size),
    title_x = title_x,
)

fig.show()

In [67]:
chart_df_5 = df.groupby(by = ['Product', 'Sales Method'], as_index = False,)[['Operating Margin']].mean()

title = 'Average Operating Margin'
#st.subheader(title)
fig = px.bar(
  chart_df_5,
  x = 'Product',
  y = 'Operating Margin',
  #text = ['{:,.2f}'.format(x) for x in chart_df_5['Operating Margin']],
  template = chart_theme,
  color = 'Sales Method',
  barmode = 'group',
  color_discrete_sequence = colors_2,
  title = title,
  text_auto = '.2%',
)
fig.update_layout(
    height = 400,
    margin = margin,
    titlefont = dict(size = title_font_size),
    title_x = title_x,
)

fig.show()

## 10 Create Dashboard
### * template : ['ggplot2', 'seaborn', 'simple_white', 'plotly', 'plotly_white', 'plotly_dark', 'presentation', 'xgridoff', 'ygridoff', 'gridon', 'none']
### * change layout https://plotly.com/python-api-reference/generated/plotly.graph_objects.Layout.html
### * more chart customization : https://plotly.com/python-api-reference/index.html

In [47]:
df.head(2)

,Retailer,Invoice Date,Region,State,Product,Price Per Unit,Units Sold,Total Sales,Operating Profit,Operating Margin,Sales Method
0,Foot Locker,2020-01-01,Northeast,New York,Men'S Street Footwear,50.0,1200,600000.0,300000.0,0.5,In-Store
1,Foot Locker,2020-01-02,Northeast,New York,Men'S Athletic Footwear,50.0,1000,500000.0,150000.0,0.3,In-Store


In [48]:
df['Product'].unique()

array(["Men'S Street Footwear", "Men'S Athletic Footwear",
       "Women'S Street Footwear", "Women'S Athletic Footwear",
       "Men'S Apparel", "Women'S Apparel"], dtype=object)

In [109]:
%%writefile app.py

import streamlit as st
import os
import pandas as pd
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import warnings
warnings.filterwarnings('ignore')

# 00 CREATING TAB TITLE

st.set_page_config(
  page_title = 'Adidas US',
  page_icon = ':bar_chart:',
  layout = 'wide',
)


# 01 CREATING DASHBOARD TITLE

st.title(':bar_chart: Adidas US Sales Dashboard')
st.markdown('<style>div,block-container{padding-top:0rem;}<style>', unsafe_allow_html = True)


# 02 IMPORTING DATASET

dataset_path = 'https://raw.githubusercontent.com/azzindani/00_Data_Source/main/Adidas_US_Sales.csv'
df = pd.read_csv(dataset_path, encoding = 'ISO-8859-1')

gdf = gpd.read_file('https://raw.githubusercontent.com/PublicaMundi/MappingAPI/master/data/geojson/us-states.json')
gdf = gdf.rename(columns = {'name' : 'State'})
gdf = gdf[['State', 'geometry']]


# 03 SETUP TEMPLATE & THEME

colors_1 = px.colors.sequential.Aggrnyl_r
colors_2 = px.colors.sequential.Magma
explode = tuple([0.015] * 50)
latitude = 57
longitude = -113
chart_theme = 'plotly_dark'
streamlit_theme = 'streamlit'
margin = {'r' : 20, 't' : 40, 'l' : 20, 'b' : 10}
cmap = 'YlGn'
title_x = 0
title_font_size = 18


# 04 CREATING DATE PICKER

col_1, col_2 = st.columns((2))
df['Invoice Date'] = pd.to_datetime(df['Invoice Date'])

# Getting min & max date
start_date = pd.to_datetime(df['Invoice Date']).min()
end_date = pd.to_datetime(df['Invoice Date']).max()

with col_1:
  date_1 = pd.to_datetime(st.date_input('Start Date', start_date))

with col_2:
  date_2 = pd.to_datetime(st.date_input('End Date', end_date))

df = df[(df['Invoice Date'] >= date_1) & (df['Invoice Date'] <= date_2)] #'''


# 05 CREATING SIDEBAR FILTER

st.sidebar.header('Choose your Filter: ')

# Create region sidebar
region = st.sidebar.multiselect('Select Region', df['Region'].unique())

if not region:
  df_2 = df.copy()
else:
  df_2 = df[df['Region'].isin(region)]

# Create state sidebar
state = st.sidebar.multiselect('Select State', df_2['State'].unique())


# 06 CREATING DATASET FILTER LOGIC

# Filter the data based on region, state

# None filter
if not region and not state:
  filtered_df = df

# Single filter
elif not state:
  filtered_df = df[df['Region'].isin(region)]

elif not region:
  filtered_df = df[df['State'].isin(state)]

else:
  filtered_df = df_2[df_2['Region'].isin(region) & df_2['State'].isin(state)]


# 07 CREATING DASHBOARD

# create highlighted indicator

col_11, col_12, col_13 = st.columns((1, 1, 3))

with col_11:
  title = 'Sales Value'
  #st.subheader(title)
  fig = go.Figure(go.Indicator(
    mode = 'number+delta',
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = filtered_df['Total Sales'].sum(),
    number = {'prefix' : '$'},
  ))

  fig.update_layout(
    #paper_bgcolor = 'lightgray',
    height = 200,
    margin = margin,
    title = title,
    titlefont = dict(size = title_font_size),
    title_x = title_x,
  )

  st.plotly_chart(fig, use_container_width = True, theme = streamlit_theme)

with col_12:
  title = 'Unit Sold'
  #st.subheader(title)
  fig = go.Figure(go.Indicator(
    mode = 'number+delta',
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = filtered_df['Units Sold'].sum(),
    number = {'valueformat' : ','},
  ))

  fig.update_layout(
    #paper_bgcolor = 'lightgray',
    height = 200,
    margin = margin,
    title = title,
    titlefont = dict(size = title_font_size),
    title_x = title_x,
  )

  st.plotly_chart(fig, use_container_width = True, theme = streamlit_theme)

# create geospatial chart

map_df = df.groupby(['State'])['Total Sales'].sum()
map_df = map_df.reset_index()
map_df = map_df.rename(columns = {'Total Sales' : 'Sales Value (USD)'})
map_df = gdf.merge(map_df, on = 'State')

var_label = 'State'
var_number = 'Sales Value (USD)'

map_df = map_df.set_index(var_label)

with col_13:
  title = 'Geospatial Sales Data'
  fig = px.choropleth_mapbox(
      data_frame = map_df,
      geojson = map_df.geometry,
      locations = map_df.index,
      color = var_number,
      color_continuous_scale = colors_1,
      #range_color = (0, 10),
      opacity = 0.5,
      center = {'lat' : latitude, 'lon' : longitude},
      mapbox_style = 'carto-positron',
      zoom = 2,
  )

  fig.update_geos(
      fitbounds = 'locations',
      visible = False,
  )
  fig.update_layout(
      height = 600,
      margin = margin,
      title = title,
      titlefont = dict(size = title_font_size),
      title_x = title_x,
  )
  st.plotly_chart(fig, use_container_width = True, theme = streamlit_theme)

# create market outlook chart

# create bar chart

chart_df_1 = filtered_df.groupby(by = 'Retailer', as_index = False,)[['Total Sales']].sum()

with col_11:
  title = 'Sales Value by Retailer'
  fig = px.bar(
    chart_df_1,
    y = 'Retailer',
    x = 'Total Sales',
    title = title,
    color_discrete_sequence = colors_1,
    text_auto = '$,.0f',
  )

  fig.update_traces(
      textfont_size = 12,
      textangle = 0,
      textposition = 'inside',
      cliponaxis = False
  )

  fig.update_xaxes(showspikes = True)
  fig.update_yaxes(showspikes = True)
  fig.update_layout(
    height = 300,
    margin = margin,
    title = title,
    titlefont = dict(size = title_font_size),
    title_x = title_x,
  )
  st.plotly_chart(fig, use_container_width = True, theme = streamlit_theme)

  with st.expander('View Data'):
    st.write(chart_df_1.style.background_gradient(cmap = cmap))
    csv = chart_df_1.to_csv(index = False).encode('utf-8')
    st.download_button('Download Data', data = csv, file_name = title + '.csv', mime = 'text/csv', help = 'Click here to download as CSV file')

chart_df_2 = filtered_df.groupby(by = 'Sales Method', as_index = False,)[['Total Sales']].sum()

with col_12:
  title = 'Sales Value by Sales Method'
  fig = px.bar(
    chart_df_2,
    y = 'Sales Method',
    x = 'Total Sales',
    title = title,
    color_discrete_sequence = colors_1,
    text_auto = '$,.0f',
  )

  fig.update_traces(
      textfont_size = 12,
      textangle = 0,
      textposition = 'inside',
      cliponaxis = False
  )

  fig.update_xaxes(showspikes = True)
  fig.update_yaxes(showspikes = True)
  fig.update_layout(
    height = 300,
    margin = margin,
    title = title,
    titlefont = dict(size = title_font_size),
    title_x = title_x,
  )
  st.plotly_chart(fig, use_container_width = True, theme = streamlit_theme)

  with st.expander('View Data'):
    st.write(chart_df_2.style.background_gradient(cmap = cmap))
    csv = chart_df_2.to_csv(index = False).encode('utf-8')
    st.download_button('Download Data', data = csv, file_name = title + '.csv', mime = 'text/csv', help = 'Click here to download as CSV file')

col_21, col_22, col_23 = st.columns((1, 1, 1))

chart_df_3 = filtered_df.groupby(by = ['Region'], as_index = False,)[['Total Sales']].sum()

with col_21:
  title = 'Sales Ration by Region'
  #st.subheader(title)
  fig = px.pie(
    chart_df_3,
    values = 'Total Sales',
    names = 'Region',
    hole = 0.5,
    template = chart_theme,
    color_discrete_sequence = colors_1,
  )
  fig.update_traces(
    text = chart_df_3['Region'],
    textposition = 'inside',
    pull = explode,
  )
  fig.update_layout(
    height = 400,
    margin = margin,
    title = title,
    titlefont = dict(size = title_font_size),
    title_x = title_x,
  )
  st.plotly_chart(fig, use_container_width = True, theme = streamlit_theme)

  with st.expander('View Data'):
    st.write(chart_df_3.style.background_gradient(cmap = cmap))
    csv = chart_df_3.to_csv(index = False).encode('utf-8')
    st.download_button('Download Data', data = csv, file_name = title + '.csv', mime = 'text/csv', help = 'Click here to download as CSV file')

chart_df_4 = filtered_df.groupby(by = ['Product'], as_index = False,)[['Total Sales', 'Operating Profit']].sum()

with col_22:
  title = 'Sales & Operating Profit by Product'
  #st.subheader(title)
  fig = go.Figure(data = [
    go.Bar(
      name = 'Total Sales',
      x = chart_df_4['Product'],
      y = chart_df_4['Total Sales'],
      text = ['${:,.0f}'.format(x) for x in chart_df_4['Total Sales']],
      marker = {'color': colors_1[0]}),
    go.Bar(
      name = 'Operating Margin',
      x = chart_df_4['Product'],
      y = chart_df_4['Operating Profit'],
      text = ['${:,.0f}'.format(x) for x in chart_df_4['Operating Profit']],
      marker = {'color': colors_1[1]}),
  ])

  fig.update_layout(
      template = chart_theme,
      #paper_bgcolor = 'LightSteelBlue',
      height = 400,
      margin = margin,
      title = title,
      titlefont = dict(size = title_font_size),
      title_x = title_x,
  )
  st.plotly_chart(fig, use_container_width = True, theme = streamlit_theme)

  with st.expander('View Data'):
    st.write(chart_df_4.style.background_gradient(cmap = cmap))
    csv = chart_df_4.to_csv(index = False).encode('utf-8')
    st.download_button('Download Data', data = csv, file_name = title + '.csv', mime = 'text/csv', help = 'Click here to download as CSV file')

chart_df_5 = filtered_df.groupby(by = ['Product', 'Sales Method'], as_index = False,)[['Operating Margin']].mean()

with col_23:
  title = 'Average Operating Margin'
  #st.subheader(title)
  fig = px.bar(
    chart_df_5,
    x = 'Product',
    y = 'Operating Margin',
    #text = ['{:,.2f}'.format(x) for x in chart_df_5['Operating Margin']],
    template = chart_theme,
    color = 'Sales Method',
    barmode = 'group',
    color_discrete_sequence = colors_1,
    title = title,
    text_auto = '.2%',
  )

  fig.update_xaxes(visible = True, showspikes = False)
  fig.update_yaxes(visible = True, showspikes = False)

  fig.update_layout(
      height = 400,
      margin = margin,
      titlefont = dict(size = title_font_size),
      title_x = title_x,
  )
  st.plotly_chart(fig, use_container_width = True, theme = streamlit_theme)

  with st.expander('View Data'):
    st.write(chart_df_5.style.background_gradient(cmap = cmap))
    csv = chart_df_5.to_csv(index = False).encode('utf-8')
    st.download_button('Download Data', data = csv, file_name = title + '.csv', mime = 'text/csv', help = 'Click here to download as CSV file')

chart_df_6 = filtered_df.groupby(by = 'Product', as_index = False,)[['Total Sales']].sum()

col_31, col_32 = st.columns((1, 1))

with col_31:
  title = 'Product Sales Value (USD)'
  fig = px.bar(
    chart_df_6,
    x = 'Product',
    y = 'Total Sales',
    title = title,
    color_discrete_sequence = colors_1,
    text_auto = '$,.0f',
  )

  fig.update_traces(
      textfont_size = 12,
      textangle = 0,
      textposition = 'inside',
      cliponaxis = False
  )

  fig.update_xaxes(showspikes = True)
  fig.update_yaxes(showspikes = True)
  fig.update_layout(
    height = 400,
    margin = margin,
    title = title,
    titlefont = dict(size = title_font_size),
    title_x = title_x,
  )
  st.plotly_chart(fig, use_container_width = True, theme = streamlit_theme)

  with st.expander('View Data'):
    st.write(chart_df_6.style.background_gradient(cmap = cmap))
    csv = chart_df_6.to_csv(index = False).encode('utf-8')
    st.download_button('Download Data', data = csv, file_name = title + '.csv', mime = 'text/csv', help = 'Click here to download as CSV file')

filtered_df['Month & Year'] = filtered_df['Invoice Date'].dt.to_period('M')
chart_df_7 = pd.DataFrame(filtered_df.groupby([filtered_df['Month & Year'], 'Product'])[['Units Sold']].mean()).reset_index()
chart_df_7['Month & Year'] = chart_df_7['Month & Year'].astype(str)

with col_32:
  title = 'Amount of Product Sold'
  fig = px.line(
    chart_df_7,
    x = 'Month & Year',
    y = 'Units Sold',
    color = 'Product',
    color_discrete_sequence = colors_1,
  )

  fig.update_traces(
      textposition = 'bottom right'
  )

  fig.update_traces(mode = 'markers+lines', hovertemplate = None)
  fig.update_layout(hovermode = 'x')

  fig.update_xaxes(showspikes = True)
  fig.update_yaxes(showspikes = True)
  fig.update_layout(
    height = 400,
    margin = margin,
    title = title,
    titlefont = dict(size = title_font_size),
    title_x = title_x,
  )
  st.plotly_chart(fig, use_container_width = True, theme = streamlit_theme)

  with st.expander('View Data'):
    st.write(chart_df_7.style.background_gradient(cmap = cmap))
    csv = chart_df_7.to_csv(index = False).encode('utf-8')
    st.download_button('Download Data', data = csv, file_name = title + '.csv', mime = 'text/csv', help = 'Click here to download as CSV file')

title = 'Relationship between Product Price and Unit Sold using Scatter Plot'
fig_1 = px.scatter(
  filtered_df,
  x = 'Price per Unit',
  y = 'Units Sold',
  color = 'Product',
  height = 600,
  template = chart_theme,
  color_discrete_sequence = colors_1,
  trendline = 'ols',
  marginal_x = 'histogram',
  marginal_y = 'box',
)
fig_1.update_layout(
  title = title,
  titlefont = dict(size = title_font_size),
  xaxis = dict(title = 'Price Per Unit (USD)', titlefont = dict(size = 14), tickformat = '.2s'),
  yaxis = dict(title = 'Units Sold', titlefont = dict(size = 14)),
  title_x = title_x,
)
st.plotly_chart(fig_1, use_container_width = True, theme = streamlit_theme)

# Download original sample dataset

with st.expander('Sample Data'):
  st.write(df.iloc[:500, 1:20:2].style.background_gradient(cmap = cmap))
  csv = df.to_csv(index = False).encode('utf-8')
  st.download_button('Download Data', data = csv, file_name = 'Data.csv', mime = 'text/csv')

Overwriting app.py


## 11 Run Dashboard

In [110]:
!streamlit run app.py &>/content/logs.txt & curl ipv4.icanhazip.com
!npx localtunnel --port 8501

34.73.191.19
npx: installed 22 in 2.358s
your url is: https://khaki-geckos-thank.loca.lt
^C
